In [1]:
# %cd "/content/drive/MyDrive/인공지능/pytorch/data"
# !unzip cornell_movie-dialogs_corpus.zip

## 1.세팅

In [2]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from __future__ import unicode_literals

import torch
from torch.jit import script, trace
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import csv
import random
import re
import os
import unicodedata
import codecs
from io import open
import itertools
import math

USE_CUDA = torch.cuda.is_available()
device = torch.device('cuda' if USE_CUDA else "cpu")

# 2.데이터 읽기 & 전처리 하기

In [3]:
## 데이터 불러오기 및 읽기

corpus_name = 'cornell movie-dialogs corpus'
corpus = os.path.join("/content/drive/MyDrive/인공지능/pytorch/data", corpus_name)

def printLines(file, n=10):
  with open(file, 'rb') as datafile:
    lines = datafile.readlines()
  for line in lines[:n]:
    print(line)

printLines(os.path.join(corpus, 'movie_lines.txt'))

b'L1045 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ They do not!\n'
b'L1044 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ They do to!\n'
b'L985 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ I hope so.\n'
b'L984 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ She okay?\n'
b"L925 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ Let's go.\n"
b'L924 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ Wow\n'
b"L872 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ Okay -- you're gonna need to learn how to lie.\n"
b'L871 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ No\n'
b'L870 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ I\'m kidding.  You know how sometimes you just become this "persona"?  And you don\'t know how to quit?\n'
b'L869 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ Like my fear of wearing pastels?\n'


In [4]:
## 원하는 형식의 데이터 파일로 만들기
'''
우리의 편의를 위해서 데이터 형식을 원하는 형태로 만든다.
각 줄에 질의 문장과 응답 문장의 쌍이 탭으로 구분되어 있게끔 하는 것~!
'''

## 파일에 포함된 대사를 쪼개서 항목에 대한 사전(``dict``) 형태로 변환
def loadLines(fileName, fields):
    lines = {}
    with open(fileName, 'r', encoding='iso-8859-1') as f:
        for line in f:
            values = line.split(" +++$+++ ")
            ## 항목을 추출
            lineObj = {}
            for i, field in enumerate(fields):
                lineObj[field] = values[i]
            lines[lineObj['lineID']] = lineObj
    return lines


## 대사의 항목을 *movie_conversations.txt* 를 참고하여 대화 형태로 묶기
def loadConversations(fileName, lines, fields):
    conversations = []
    with open(fileName, 'r', encoding='iso-8859-1') as f:
        for line in f:
            values = line.split(" +++$+++ ")
            ## 항목을 추출
            convObj = {}
            for i, field in enumerate(fields):
                convObj[field] = values[i]
            ## 문자열을 리스트로 변환 -> ex. (convObj["utteranceIDs"] == "['L598485', 'L598486', ...]")
            utterance_id_pattern = re.compile('L[0-9]+')
            lineIds = utterance_id_pattern.findall(convObj["utteranceIDs"])
            ## 대사를 재구성하기
            convObj["lines"] = []
            for lineId in lineIds:
                convObj["lines"].append(lines[lineId])
            conversations.append(convObj)
    return conversations


## conversations에서 문장 쌍을 추출
def extractSentencePairs(conversations):
    qa_pairs = []
    for conversation in conversations:
        ## 대화를 이루는 각 대사에 대해 반복문을 수행
        ## 대화의 마지막 대사는 (그에 대한 응답이 없으므로) 무시
        for i in range(len(conversation["lines"]) - 1):
            inputLine = conversation["lines"][i]["text"].strip()
            targetLine = conversation["lines"][i+1]["text"].strip()
            ## 잘못된 샘플은 제거 (리스트가 하나라도 비어 있는 경우)
            if inputLine and targetLine:
                qa_pairs.append([inputLine, targetLine])
    return qa_pairs

In [5]:
## 이제 위에 함수들을 호출해서 새로운 파일인 "formatted_movie_lines.txt" 를 만든다!

## 새 파일에 대한 경로를 정의
datafile = os.path.join(corpus, "formatted_movie_lines.txt")

delimiter = '\t'
## 구분자에 대해 unescape 함수를 호출
delimiter = str(codecs.decode(delimiter, "unicode_escape"))

## 대사 사전(dict), 대화 리스트(list), 그리고 각 항목의 이름을 초기화하기
lines = {}
conversations = []
MOVIE_LINES_FIELDS = ["lineID", "characterID", "movieID", "character", "text"]
MOVIE_CONVERSATIONS_FIELDS = ["character1ID", "character2ID", "movieID", "utteranceIDs"]

## 대사(lines)를 읽어들여 대화(conversations)로 재구성
print("\nProcessing corpus...")
lines = loadLines(os.path.join(corpus, "movie_lines.txt"), MOVIE_LINES_FIELDS)
print("\nLoading conversations...")
conversations = loadConversations(os.path.join(corpus, "movie_conversations.txt"),
                                  lines, MOVIE_CONVERSATIONS_FIELDS)

## 결과를 새로운 csv 파일로 저장
print("\nWriting newly formatted file...")
with open(datafile, 'w', encoding='utf-8') as outputfile:
    writer = csv.writer(outputfile, delimiter=delimiter, lineterminator='\n')
    for pair in extractSentencePairs(conversations):
        writer.writerow(pair)

## 몇 줄을 예제 삼아 출력
print("\nSample lines from file:")
printLines(datafile)


Processing corpus...

Loading conversations...

Writing newly formatted file...

Sample lines from file:
b"Can we make this quick?  Roxanne Korrine and Andrew Barrett are having an incredibly horrendous public break- up on the quad.  Again.\tWell, I thought we'd start with pronunciation, if that's okay with you.\n"
b"Well, I thought we'd start with pronunciation, if that's okay with you.\tNot the hacking and gagging and spitting part.  Please.\n"
b"Not the hacking and gagging and spitting part.  Please.\tOkay... then how 'bout we try out some French cuisine.  Saturday?  Night?\n"
b"You're asking me out.  That's so cute. What's your name again?\tForget it.\n"
b"No, no, it's my fault -- we didn't have a proper introduction ---\tCameron.\n"
b"Cameron.\tThe thing is, Cameron -- I'm at the mercy of a particularly hideous breed of loser.  My sister.  I can't date until she does.\n"
b"The thing is, Cameron -- I'm at the mercy of a particularly hideous breed of loser.  My sister.  I can't dat

In [6]:
## 데이터 읽고 정리하기
'''
이제는 어휘집을 만들고 질의/응답 문장 쌍을 메모리로 읽어들이는 것!
우리는 데이터셋 안에 들어 있는 단어를 인덱스 값으로 변환하는 매핑을 따로 만들어야 함
'''

## 기본 단어 토큰 값
PAD_token = 0  ## 짧은 문장을 채울(패딩, PADding) 때 사용할 제로 토큰
SOS_token = 1  ## 문장의 시작(SOS, Start Of Sentence)을 나타내는 토큰
EOS_token = 2  ## 문장의 끝(EOS, End Of Sentence)을 나태는 토큰

class Voc:
    def __init__(self, name):
        self.name = name
        self.trimmed = False
        self.word2index = {}
        self.word2count = {}
        self.index2word = {PAD_token: "PAD", SOS_token: "SOS", EOS_token: "EOS"}
        self.num_words = 3  ## SOS, EOS, PAD를 센 것

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.num_words
            self.word2count[word] = 1
            self.index2word[self.num_words] = word
            self.num_words += 1
        else:
            self.word2count[word] += 1

    ## 등장 횟수가 기준 이하인 단어를 정리
    def trim(self, min_count):
        if self.trimmed:
            return
        self.trimmed = True

        keep_words = []

        for k, v in self.word2count.items():
            if v >= min_count:
                keep_words.append(k)

        print('keep_words {} / {} = {:.4f}'.format(
            len(keep_words), len(self.word2index), len(keep_words) / len(self.word2index)
        ))

        ## 사전을 다시 초기화
        self.word2index = {}
        self.word2count = {}
        self.index2word = {PAD_token: "PAD", SOS_token: "SOS", EOS_token: "EOS"}
        self.num_words = 3 ## 기본 토큰을 센 것

        for word in keep_words:
            self.addWord(word)

In [7]:
'''
이제 어휘집과 질의/응답 문장 쌍을 재구성
데이터를 사용하려면 전처리 작업을 해야함

1. unicodeToAscii을 이용해서 유니코드 문자열을 아스키코드로 변환해야한다.
2. 모든 글자를 소문자로 변환하고, 알파벡도 아니고 기본적인 문장 부호도 아닌 글자는 제거 (normalizeString  <- 정규화)
3. 학습할 때 유용하기 위해서 문장의 길이가 일정 기준을 초과하면 제거 ( MAX_LENGTH를 넘게 되면 삭제! )
'''

MAX_LENGTH = 10  ## 고려할 문장의 최대 길이

## 유니코드 문자열을 아스키로 변환
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

## 소문자로 만들고, 공백을 넣고, 알파벳 외의 글자를 제거
def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    s = re.sub(r"\s+", r" ", s).strip()
    return s

## 질의/응답 쌍을 읽어서 voc 객체를 반환
def readVocs(datafile, corpus_name):
    print("Reading lines...")
    ## 파일을 읽고, 쪼개어 lines에 저장
    lines = open(datafile, encoding='utf-8').\
        read().strip().split('\n')
    ## 각 줄을 쪼개어 pairs에 저장하고 정규화
    pairs = [[normalizeString(s) for s in l.split('\t')] for l in lines]
    voc = Voc(corpus_name)
    return voc, pairs

## 문장의 쌍 'p'에 포함된 두 문장이 모두 MAX_LENGTH라는 기준보다 짧은지를 반환
def filterPair(p):
    # EOS 토큰을 위해 입력 시퀀스의 마지막 단어를 보존해야 합니다
    return len(p[0].split(' ')) < MAX_LENGTH and len(p[1].split(' ')) < MAX_LENGTH

## 조건식 filterPair에 따라 pairs를 필터링
def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

## 앞에서 정의한 함수를 이용하여 만든 voc 객체와 리스트 pairs를 반환
def loadPrepareData(corpus, corpus_name, datafile, save_dir):
    print("Start preparing training data ...")
    voc, pairs = readVocs(datafile, corpus_name)
    print("Read {!s} sentence pairs".format(len(pairs)))
    pairs = filterPairs(pairs)
    print("Trimmed to {!s} sentence pairs".format(len(pairs)))
    print("Counting words...")
    for pair in pairs:
        voc.addSentence(pair[0])
        voc.addSentence(pair[1])
    print("Counted words:", voc.num_words)
    return voc, pairs


## voc와 pairs를 읽고 재구성
save_dir = os.path.join("data", "save")
voc, pairs = loadPrepareData(corpus, corpus_name, datafile, save_dir)
## 검증을 위해 pairs의 일부 내용을 출력
print("\npairs:")
for pair in pairs[:10]:
    print(pair)

Start preparing training data ...
Reading lines...
Read 221282 sentence pairs
Trimmed to 64271 sentence pairs
Counting words...
Counted words: 18008

pairs:
['there .', 'where ?']
['you have my word . as a gentleman', 'you re sweet .']
['hi .', 'looks like things worked out tonight huh ?']
['you know chastity ?', 'i believe we share an art instructor']
['have fun tonight ?', 'tons']
['well no . . .', 'then that s all you had to say .']
['then that s all you had to say .', 'but']
['but', 'you always been this selfish ?']
['do you listen to this crap ?', 'what crap ?']
['what good stuff ?', 'the real you .']


In [8]:
'''
학습이 빠르게 끝날 수 있도록 자주 쓰이지 않은 단어를 어휘집에서 제거하기

1. voc.trim 함수를 사용해서 MIN_COUNT 기준 이하의 단어들은 제거
2. 제거하기로 한 단어를 포함하는 경우를 PAIRS에서 제거
'''

MIN_COUNT = 3 # 제외할 단어의 기준이 되는 등장 횟수

def trimRareWords(voc, pairs, MIN_COUNT):
  # MIN_COUNT 미만으로 사용된 단어는 voc에서 제외
  voc.trim(MIN_COUNT)
  # 제외할 단어가 포함된 경우를 pairs에서도 제외
  keep_pairs = []
  for pair in pairs:
    input_sentence = pair[0]
    output_sentence = pair[1]
    keep_input = True
    keep_output = True
    # 입력 문장을 검사
    for word in input_sentence.split(' '):
      if word not in voc.word2index:
        keep_input = False
        break
    # 출력 문장을 검사
    for word in output_sentence.split(' '):
      if word not in voc.word2index:
        keep_output = False
        break
    # 입출력 문장에 제외하기로한 단어를 포함하지 않는 경우만 남겨두기
    if keep_input and keep_output:
      keep_pairs.append(pair)
  
  print('Trimmed from {} pairs to {}, {:.4f} or total'.format(len(pairs), len(keep_pairs), len(keep_pairs) / len(pairs)))
  return keep_pairs

In [9]:
pairs = trimRareWords(voc, pairs, MIN_COUNT)

keep_words 7823 / 18005 = 0.4345
Trimmed from 64271 pairs to 53165, 0.8272 or total


# 3. 모델을 위한 데이터 준비하기

In [23]:
## 우리가 전처리를 했지만 우리가 모델에서 사용하는 입력은 수치 값으로 이뤄진 torch 텐서임
## 데이터를 tensor 형태로 만들어주기

def indexesFromSentence(voc, sentence):
    return [voc.word2index[word] for word in sentence.split(' ')] + [EOS_token]


def zeroPadding(l, fillvalue=PAD_token):
    return list(itertools.zip_longest(*l, fillvalue=fillvalue))

def binaryMatrix(l, value=PAD_token):
    m = []
    for i, seq in enumerate(l):
        m.append([])
        for token in seq:
            if token == PAD_token:
                m[i].append(0)
            else:
                m[i].append(1)
    return m

## 입력 시퀀스 텐서에 패딩한 결과와 lengths를 반환
def inputVar(l, voc):
    indexes_batch = [indexesFromSentence(voc, sentence) for sentence in l]
    lengths = torch.tensor([len(indexes) for indexes in indexes_batch])
    padList = zeroPadding(indexes_batch)
    padVar = torch.LongTensor(padList)
    return padVar, lengths

## 패딩한 목표 시퀀스 텐서, 패딩 마스크, 그리고 최대 목표 길이를 반환
def outputVar(l, voc):
    indexes_batch = [indexesFromSentence(voc, sentence) for sentence in l]
    max_target_len = max([len(indexes) for indexes in indexes_batch])
    padList = zeroPadding(indexes_batch)
    mask = binaryMatrix(padList)
    mask = torch.ByteTensor(mask)
    padVar = torch.LongTensor(padList)
    return padVar, mask, max_target_len

## 입력 배치를 이루는 쌍에 대한 모든 아이템을 반환
def batch2TrainData(voc, pair_batch):
    pair_batch.sort(key=lambda x: len(x[0].split(" ")), reverse=True)
    input_batch, output_batch = [], []
    for pair in pair_batch:
        input_batch.append(pair[0])
        output_batch.append(pair[1])
    inp, lengths = inputVar(input_batch, voc)
    output, mask, max_target_len = outputVar(output_batch, voc)
    return inp, lengths, output, mask, max_target_len


## 검증용 예시
small_batch_size = 5
batches = batch2TrainData(voc, [random.choice(pairs) for _ in range(small_batch_size)])
input_variable, lengths, target_variable, mask, max_target_len = batches

print("input_variable:", input_variable)
print("lengths:", lengths)
print("target_variable:", target_variable)
print("mask:", mask)
print("max_target_len:", max_target_len)

input_variable: tensor([[  50,   25,   34, 2530,    5],
        [  92,    8,  625,   66,    6],
        [   7,   12,    4,    2,    2],
        [ 206, 4971,    2,    0,    0],
        [   4,   40,    0,    0,    0],
        [   4, 2037,    0,    0,    0],
        [   4,    4,    0,    0,    0],
        [   2,    2,    0,    0,    0]])
lengths: tensor([8, 8, 4, 3, 3])
target_variable: tensor([[  25,  150,  490,  124,   53],
        [  89,  410,    6,   16, 3802],
        [ 534,  542,    2, 1119,    4],
        [  40,    4,    0,    4,    4],
        [1419,    2,    0,    2,    4],
        [  76,    0,    0,    0,    2],
        [   4,    0,    0,    0,    0],
        [   2,    0,    0,    0,    0]])
mask: tensor([[1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1],
        [1, 1, 0, 1, 1],
        [1, 1, 0, 1, 1],
        [1, 0, 0, 0, 1],
        [1, 0, 0, 0, 0],
        [1, 0, 0, 0, 0]], dtype=torch.uint8)
max_target_len: 8


# 4. 모델 정의하기

In [24]:
class EncoderRNN(nn.Module):
    def __init__(self, hidden_size, embedding, n_layers=1, dropout=0):
        super(EncoderRNN, self).__init__()
        self.n_layers = n_layers
        self.hidden_size = hidden_size
        self.embedding = embedding

        '''
        GRU를 초기화,  input_size와 hidden_size 매개변수는 둘 다 'hidden_size'로 두기
        이는 입력의 크기가 hideen_size 만큼의 피처를 갖는 단어 임베딩이기 때문~
        '''

        self.gru = nn.GRU(hidden_size, hidden_size, n_layers,
                          dropout=(0 if n_layers == 1 else dropout), bidirectional=True)

    def forward(self, input_seq, input_lengths, hidden=None):
        ## 단어 인덱스를 임베딩으로 변환
        embedded = self.embedding(input_seq)
        ## RNN 모듈을 위한 패딩된 배치 시퀀스를 패킹
        packed = nn.utils.rnn.pack_padded_sequence(embedded, input_lengths)
        ## GRU로 포워드 패스를 수행
        outputs, hidden = self.gru(packed, hidden)
        ## 패딩을 언패킹
        outputs, _ = nn.utils.rnn.pad_packed_sequence(outputs)
        ## 양방향 GRU의 출력을 합산
        outputs = outputs[:, :, :self.hidden_size] + outputs[:, : ,self.hidden_size:]
        ## 출력과 마지막 은닉 상태를 반환
        return outputs, hidden

In [25]:
## Luong 어텐션 레이어
class Attn(nn.Module):
    def __init__(self, method, hidden_size):
        super(Attn, self).__init__()
        self.method = method
        if self.method not in ['dot', 'general', 'concat']:
            raise ValueError(self.method, "is not an appropriate attention method.")
        self.hidden_size = hidden_size
        if self.method == 'general':
            self.attn = nn.Linear(self.hidden_size, hidden_size)
        elif self.method == 'concat':
            self.attn = nn.Linear(self.hidden_size * 2, hidden_size)
            self.v = nn.Parameter(torch.FloatTensor(hidden_size))

    def dot_score(self, hidden, encoder_output):
        return torch.sum(hidden * encoder_output, dim=2)

    def general_score(self, hidden, encoder_output):
        energy = self.attn(encoder_output)
        return torch.sum(hidden * energy, dim=2)

    def concat_score(self, hidden, encoder_output):
        energy = self.attn(torch.cat((hidden.expand(encoder_output.size(0), -1, -1), encoder_output), 2)).tanh()
        return torch.sum(self.v * energy, dim=2)

    def forward(self, hidden, encoder_outputs):
        ## Attention 가중치(에너지)를 제안된 방법에 따라 계산
        if self.method == 'general':
            attn_energies = self.general_score(hidden, encoder_outputs)
        elif self.method == 'concat':
            attn_energies = self.concat_score(hidden, encoder_outputs)
        elif self.method == 'dot':
            attn_energies = self.dot_score(hidden, encoder_outputs)

        ## max_length와 batch_size의 차원을 뒤집기
        attn_energies = attn_energies.t()

        ## 정규화된 softmax 확률 점수를 반환 (차원을 늘려서)
        return F.softmax(attn_energies, dim=1).unsqueeze(1)

In [26]:
class LuongAttnDecoderRNN(nn.Module):
    def __init__(self, attn_model, embedding, hidden_size, output_size, n_layers=1, dropout=0.1):
        super(LuongAttnDecoderRNN, self).__init__()

        ## 참조를 보존
        self.attn_model = attn_model
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers
        self.dropout = dropout

        ## 레이어를 정의
        self.embedding = embedding
        self.embedding_dropout = nn.Dropout(dropout)
        self.gru = nn.GRU(hidden_size, hidden_size, n_layers, dropout=(0 if n_layers == 1 else dropout))
        self.concat = nn.Linear(hidden_size * 2, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)

        self.attn = Attn(attn_model, hidden_size)

    def forward(self, input_step, last_hidden, encoder_outputs):
        ## 주의: 한 단위 시간에 대해 한 단계(단어)만을 수행
        ## 현재의 입력 단어에 대한 임베딩을 구하기
        embedded = self.embedding(input_step)
        embedded = self.embedding_dropout(embedded)
        ## 무방향 GRU로 포워드 패스를 수행
        rnn_output, hidden = self.gru(embedded, last_hidden)
        ## 현재의 GRU 출력을 바탕으로 어텐션 가중치를 계산
        attn_weights = self.attn(rnn_output, encoder_outputs)
        ## 인코더 출력에 어텐션을 곱하여 새로운 "가중치 합" 문맥 벡터를 구하기
        context = attn_weights.bmm(encoder_outputs.transpose(0, 1))
        ## Luong의 논문에 나온 식 5를 이용하여 가중치 문맥 벡터와 GRU 출력을 결합
        rnn_output = rnn_output.squeeze(0)
        context = context.squeeze(1)
        concat_input = torch.cat((rnn_output, context), 1)
        concat_output = torch.tanh(self.concat(concat_input))
        ## Luong의 논문에 나온 식 6을 이용하여 다음 단어를 예측
        output = self.out(concat_output)
        output = F.softmax(output, dim=1)
        ## 출력과 마지막 은닉 상태를 반환
        return output, hidden

## 5. 학습과정 정의하기

In [27]:
def maskNLLLoss(inp, target, mask):
    nTotal = mask.sum()
    crossEntropy = -torch.log(torch.gather(inp, 1, target.view(-1, 1)).squeeze(1))
    loss = crossEntropy.masked_select(mask).mean()
    loss = loss.to(device)
    return loss, nTotal.item()

In [28]:
def train(input_variable, lengths, target_variable, mask, max_target_len, encoder, decoder, embedding,
          encoder_optimizer, decoder_optimizer, batch_size, clip, max_length=MAX_LENGTH):

    ## zero_grad -> 가중치 초기화
    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    ## device 옵션을 설정
    input_variable = input_variable.to(device)
    target_variable = target_variable.to(device)
    mask = mask.to(device)
    lengths = lengths.to("cpu")

    ## 변수를 초기화
    loss = 0
    print_losses = []
    n_totals = 0

    ## 인코더로 포워드 패스를 수행
    encoder_outputs, encoder_hidden = encoder(input_variable, lengths)

    ## 초기 디코더 입력을 생성 (각 문장을 SOS 토큰으로 시작)
    decoder_input = torch.LongTensor([[SOS_token for _ in range(batch_size)]])
    decoder_input = decoder_input.to(device)

    ## 디코더의 초기 은닉 상태를 인코더의 마지막 은닉 상태로 두기
    decoder_hidden = encoder_hidden[:decoder.n_layers]

    ## 이번 반복에서 teacher forcing을 사용할지를 결정
    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    ## 배치 시퀀스를 한 번에 하나씩 디코더로 포워드 패스
    if use_teacher_forcing:
        for t in range(max_target_len):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden, encoder_outputs
            )
            ## Teacher forcing 사용: 다음 입력을 현재의 목표로 두기
            decoder_input = target_variable[t].view(1, -1)
            ## 손실을 계산하고 누적
            mask_loss, nTotal = maskNLLLoss(decoder_output, target_variable[t], mask[t])
            loss += mask_loss
            print_losses.append(mask_loss.item() * nTotal)
            n_totals += nTotal
    else:
        for t in range(max_target_len):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden, encoder_outputs
            )
            ## Teacher forcing 미사용: 다음 입력을 디코더의 출력으로 두기
            _, topi = decoder_output.topk(1)
            decoder_input = torch.LongTensor([[topi[i][0] for i in range(batch_size)]])
            decoder_input = decoder_input.to(device)
            ## 손실을 계산하고 누적
            mask_loss, nTotal = maskNLLLoss(decoder_output, target_variable[t], mask[t])
            loss += mask_loss
            print_losses.append(mask_loss.item() * nTotal)
            n_totals += nTotal

    ## 역전파를 수행
    loss.backward()

    ## 그라디언트 클리핑: 그라디언트를 제자리에서 수정
    _ = nn.utils.clip_grad_norm_(encoder.parameters(), clip)
    _ = nn.utils.clip_grad_norm_(decoder.parameters(), clip)

    ## 모델의 가중치를 수정
    encoder_optimizer.step()
    decoder_optimizer.step()

    return sum(print_losses) / n_totals

In [29]:
def trainIters(model_name, voc, pairs, encoder, decoder, encoder_optimizer, decoder_optimizer, embedding, encoder_n_layers, decoder_n_layers, save_dir, n_iteration, batch_size, print_every, save_every, clip, corpus_name, loadFilename):

    ## 각 단계에 대한 배치를 읽어오기
    training_batches = [batch2TrainData(voc, [random.choice(pairs) for _ in range(batch_size)])
                      for _ in range(n_iteration)]

    ## 초기화
    print('Initializing ...')
    start_iteration = 1
    print_loss = 0
    if loadFilename:
        start_iteration = checkpoint['iteration'] + 1

    ## 학습 루프
    print("Training...")
    for iteration in range(start_iteration, n_iteration + 1):
        training_batch = training_batches[iteration - 1]
        ## 배치에서 각 필드를 읽어오기
        input_variable, lengths, target_variable, mask, max_target_len = training_batch

        ## 배치에 대해 학습을 한 단계 진행
        loss = train(input_variable, lengths, target_variable, mask, max_target_len, encoder,
                     decoder, embedding, encoder_optimizer, decoder_optimizer, batch_size, clip)
        print_loss += loss

        ## 경과를 출력
        if iteration % print_every == 0:
            print_loss_avg = print_loss / print_every
            print("Iteration: {}; Percent complete: {:.1f}%; Average loss: {:.4f}".format(iteration, iteration / n_iteration * 100, print_loss_avg))
            print_loss = 0

        ## Checkpoint를 저장
        if (iteration % save_every == 0):
            directory = os.path.join(save_dir, model_name, corpus_name, '{}-{}_{}'.format(encoder_n_layers, decoder_n_layers, hidden_size))
            if not os.path.exists(directory):
                os.makedirs(directory)
            torch.save({
                'iteration': iteration,
                'en': encoder.state_dict(),
                'de': decoder.state_dict(),
                'en_opt': encoder_optimizer.state_dict(),
                'de_opt': decoder_optimizer.state_dict(),
                'loss': loss,
                'voc_dict': voc.__dict__,
                'embedding': embedding.state_dict()
            }, os.path.join(directory, '{}_{}.tar'.format(iteration, 'checkpoint')))

# 6.평가정의하기

In [30]:
class GreedySearchDecoder(nn.Module):
    def __init__(self, encoder, decoder):
        super(GreedySearchDecoder, self).__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, input_seq, input_length, max_length):
        ## 인코더 모델로 입력을 포워드 패스
        encoder_outputs, encoder_hidden = self.encoder(input_seq, input_length)
        ## 인코더의 마지막 은닉 레이어가 디코더의 첫 번째 은닉 레이어의 입력이 되도록 준비
        decoder_hidden = encoder_hidden[:decoder.n_layers]
        ## 디코더의 첫 번째 입력을 SOS_token으로 초기화
        decoder_input = torch.ones(1, 1, device=device, dtype=torch.long) * SOS_token
        ## 디코더가 단어를 덧붙여 나갈 텐서를 초기화
        all_tokens = torch.zeros([0], device=device, dtype=torch.long)
        all_scores = torch.zeros([0], device=device)
        ## 반복적으로 각 단계마다 하나의 단어 토큰을 디코딩
        for _ in range(max_length):
            ## 디코더로의 포워드 패스를 수행
            decoder_output, decoder_hidden = self.decoder(decoder_input, decoder_hidden, encoder_outputs)
            ## 가장 가능성 높은 단어 토큰과 그 softmax 점수를 구하기
            decoder_scores, decoder_input = torch.max(decoder_output, dim=1)
            ## 토큰과 점수를 기록
            all_tokens = torch.cat((all_tokens, decoder_input), dim=0)
            all_scores = torch.cat((all_scores, decoder_scores), dim=0)
            ## 현재의 토큰을 디코더의 다음 입력으로 준비 (차원을 증가시켜서)
            decoder_input = torch.unsqueeze(decoder_input, 0)
        ## 단어 토큰과 점수를 모아서 반환
        return all_tokens, all_scores

In [31]:
def evaluate(encoder, decoder, searcher, voc, sentence, max_length=MAX_LENGTH):
    ### 입력 시퀀스를 배치 형태로 만들기
    ## 단어 -> 인덱스
    indexes_batch = [indexesFromSentence(voc, sentence)]
    ## lengths 텐서를 만들기
    lengths = torch.tensor([len(indexes) for indexes in indexes_batch])
    ## 배치의 차원을 뒤집어서 모델이 사용하는 형태로 만들기
    input_batch = torch.LongTensor(indexes_batch).transpose(0, 1)
    ## 적절한 디바이스를 사용
    input_batch = input_batch.to(device)
    lengths = lengths.to("cpu")
    ## searcher를 이용하여 문장을 디코딩
    tokens, scores = searcher(input_batch, lengths, max_length)
    ## 인덱스 -> 단어
    decoded_words = [voc.index2word[token.item()] for token in tokens]
    return decoded_words


def evaluateInput(encoder, decoder, searcher, voc):
    input_sentence = ''
    while(1):
        try:
            ## 입력 문장을 받아오기
            input_sentence = input('> ')
            ## 종료 조건인지 검사
            if input_sentence == 'q' or input_sentence == 'quit': break
            ## 문장을 정규화
            input_sentence = normalizeString(input_sentence)
            ## 문장을 평가
            output_words = evaluate(encoder, decoder, searcher, voc, input_sentence)
            ## 응답 문장을 형식에 맞춰 출력
            output_words[:] = [x for x in output_words if not (x == 'EOS' or x == 'PAD')]
            print('Bot:', ' '.join(output_words))

        except KeyError:
            print("Error: Encountered unknown word.")

In [32]:
## 모델을 설정
model_name = 'cb_model'
attn_model = 'dot'
hidden_size = 500
encoder_n_layers = 2
decoder_n_layers = 2
dropout = 0.1
batch_size = 64

## 불러올 checkpoint를 설정,  처음부터 시작할 때는 None으로 두면 됨
loadFilename = None
checkpoint_iter = 4000


## loadFilename이 제공되는 경우에는 모델을 불러오기
if loadFilename:
    ## 모델을 학습할 때와 같은 기기에서 불러오는 경우
    checkpoint = torch.load(loadFilename)
    ## GPU에서 학습한 모델을 CPU로 불러오는 경우
    ## checkpoint = torch.load(loadFilename, map_location=torch.device('cpu'))
    encoder_sd = checkpoint['en']
    decoder_sd = checkpoint['de']
    encoder_optimizer_sd = checkpoint['en_opt']
    decoder_optimizer_sd = checkpoint['de_opt']
    embedding_sd = checkpoint['embedding']
    voc.__dict__ = checkpoint['voc_dict']


print('Building encoder and decoder ...')
## 단어 임베딩을 초기화
embedding = nn.Embedding(voc.num_words, hidden_size)
if loadFilename:
    embedding.load_state_dict(embedding_sd)
## 인코더 및 디코더 모델을 초기화
encoder = EncoderRNN(hidden_size, embedding, encoder_n_layers, dropout)
decoder = LuongAttnDecoderRNN(attn_model, embedding, hidden_size, voc.num_words, decoder_n_layers, dropout)
if loadFilename:
    encoder.load_state_dict(encoder_sd)
    decoder.load_state_dict(decoder_sd)
## 적절한 디바이스를 사용
encoder = encoder.to(device)
decoder = decoder.to(device)
print('Models built and ready to go!')

Building encoder and decoder ...
Models built and ready to go!


# 모델실행하기

In [33]:
'''
학습하기 -> 총 4000 iteration

먼저 학습 매개변수를 설정하고, optimize를 초기화하고, 마지막으로 trainIters 함수를 호출하여 학습 단계를 진행!
'''

## 학습 및 최적화 설정
clip = 50.0
teacher_forcing_ratio = 1.0
learning_rate = 0.0001
decoder_learning_ratio = 5.0
n_iteration = 4000
print_every = 1
save_every = 500

## Dropout 레이어를 학습 모드로 두기
encoder.train()
decoder.train()

## Optimizer를 초기화
print('Building optimizers ...')
encoder_optimizer = optim.Adam(encoder.parameters(), lr=learning_rate)
decoder_optimizer = optim.Adam(decoder.parameters(), lr=learning_rate * decoder_learning_ratio)
if loadFilename:
    encoder_optimizer.load_state_dict(encoder_optimizer_sd)
    decoder_optimizer.load_state_dict(decoder_optimizer_sd)

## cuda가 있다면 cuda를 설정
for state in encoder_optimizer.state.values():
    for k, v in state.items():
        if isinstance(v, torch.Tensor):
            state[k] = v.cuda()

for state in decoder_optimizer.state.values():
    for k, v in state.items():
        if isinstance(v, torch.Tensor):
            state[k] = v.cuda()

## 학습 단계를 수행
print("Starting Training!")
trainIters(model_name, voc, pairs, encoder, decoder, encoder_optimizer, decoder_optimizer,
           embedding, encoder_n_layers, decoder_n_layers, save_dir, n_iteration, batch_size,
           print_every, save_every, clip, corpus_name, loadFilename)

Building optimizers ...
Starting Training!
Initializing ...
Training...


<ipython-input-27-f319a975feb5>:4: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead. (Triggered internally at ../aten/src/ATen/native/IndexingUtils.h:27.)
  loss = crossEntropy.masked_select(mask).mean()
/usr/local/lib/python3.8/dist-packages/torch/autograd/__init__.py:197: UserWarning: masked_scatter_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead. (Triggered internally at ../aten/src/ATen/native/cuda/IndexKernel.cpp:74.)
  Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


Iteration: 1; Percent complete: 0.0%; Average loss: 8.9698
Iteration: 2; Percent complete: 0.1%; Average loss: 8.8601
Iteration: 3; Percent complete: 0.1%; Average loss: 8.7025
Iteration: 4; Percent complete: 0.1%; Average loss: 8.3747
Iteration: 5; Percent complete: 0.1%; Average loss: 7.9386
Iteration: 6; Percent complete: 0.1%; Average loss: 7.4582
Iteration: 7; Percent complete: 0.2%; Average loss: 6.9636
Iteration: 8; Percent complete: 0.2%; Average loss: 6.7145
Iteration: 9; Percent complete: 0.2%; Average loss: 6.7526
Iteration: 10; Percent complete: 0.2%; Average loss: 6.5410
Iteration: 11; Percent complete: 0.3%; Average loss: 6.2484
Iteration: 12; Percent complete: 0.3%; Average loss: 5.9368
Iteration: 13; Percent complete: 0.3%; Average loss: 5.6442
Iteration: 14; Percent complete: 0.4%; Average loss: 5.6275
Iteration: 15; Percent complete: 0.4%; Average loss: 5.5892
Iteration: 16; Percent complete: 0.4%; Average loss: 5.3574
Iteration: 17; Percent complete: 0.4%; Average lo

In [35]:
## 평가해보기 -> 내가 만든 챗봇을 직접 실행하면서 평가해보기

## Dropout 레이어를 평가 모드로 설정
encoder.eval()
decoder.eval()

## 탐색 모듈을 초기화
searcher = GreedySearchDecoder(encoder, decoder)

In [36]:
## 우리가 만든 챗봇 실행해보기

evaluateInput(encoder, decoder, searcher, voc)

> hi
Bot: hi . . . . ?
> hello
Bot: hello . . . . .
> your name?
Bot: yes . . . . .
> i'm good
Bot: you re not a good thing .
> you loved
Bot: you re not a victim . .
> good night
Bot: good night . . . .
> 
Bot: what ? ? ? ? ?
> by
Bot: i m not going to see you . .


KeyboardInterrupt: ignored